In [1]:
import requests
from bs4 import BeautifulSoup

#url = raw_input("Enter a website to extract the URL's from: ")
#r  = requests.get("http://" +url)
#data = r.text
#soup = BeautifulSoup(data)
#for link in soup.find_all('a'):
#    print(link.get('href'))

In [95]:
import datetime

url_dir = []


root = "http://www.comichron.com/monthlycomicssales"
year = 1995
while year < 2018:
    year_str = str(year)
    month = 1
    month_cap = 13
    if year == 2017:
        month_cap = 9
    while month < month_cap:
        month_str = str(month)
        if len(month_str) == 1:
            month_str = '0'+month_str
        if ((year < 1996) or ((year < 1997) and (month < 9))):
            month_str = month_str+'Diamond'  
        page = year_str+"-"+month_str+'.html'
        address = root + '/' + year_str + '/' + page
        date = datetime.date(year,month,1)
        new_row = (date,address)
        url_dir.append(new_row)
        month = month +1
    year = year +1

In [100]:
for x in url_dir[93:100]:
    print x

(datetime.date(2002, 10, 1), 'http://www.comichron.com/monthlycomicssales/2002/2002-10.html')
(datetime.date(2002, 11, 1), 'http://www.comichron.com/monthlycomicssales/2002/2002-11.html')
(datetime.date(2002, 12, 1), 'http://www.comichron.com/monthlycomicssales/2002/2002-12.html')
(datetime.date(2003, 1, 1), 'http://www.comichron.com/monthlycomicssales/2003/2003-01.html')
(datetime.date(2003, 2, 1), 'http://www.comichron.com/monthlycomicssales/2003/2003-02.html')
(datetime.date(2003, 3, 1), 'http://www.comichron.com/monthlycomicssales/2003/2003-03.html')
(datetime.date(2003, 4, 1), 'http://www.comichron.com/monthlycomicssales/2003/2003-04.html')


In [138]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
    
    def parse_url_share(self, url):
        r  = requests.get(url)
        data = r.text
        soup = BeautifulSoup(data)
        ta = soup.find("table", { "class" : "comichron-sharetable" })
        if ta is None:
            ta = soup.find("table", {"style" : "width: 189px; height: 2707px;"})
            if ta is None:
                ta = soup.find("table", {"style" : "height: 2707px; width: 189px;"})
                if ta is None:
                    ta = soup.find("table", {"class" : "t1"})
                    if ta is None:
                        print "returning None"
                        return None
        print ta
        return self.parse_html_table(ta)
    
    def parse_url_man(self,url):
        r  = requests.get(url)
        data = r.text
        soup = BeautifulSoup(data)
        table = soup.find( "table", {"id":"Top300Comics"} )
        if table is None:
            tables = soup.findAll("table")
            table = tables[1]
        return self.parse_html_table(table,head=False)
    
    def parse_url(self, url):
        r  = requests.get(url)
        data = r.text
        soup = BeautifulSoup(data)
        table = soup.find( "table", {"id":"Top300Comics"} )
        if table is None:
            tables = soup.findAll("table")
            table = tables[1]
        return self.parse_html_table(table)
    
    def parse_html_table(self, table, head=True):
        n_columns = 0
        n_rows=0
        column_names = []
    
        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns < len(td_tags):
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                        
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
#            raise Exception("Column titles do not match the number of columns")
            print "Mismatched cols and names"
            i = 0
            while i < n_columns:
                try:
                    column_names[i]
                except:
                    column_names.append("")
                i = i+1
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        
        df = pd.DataFrame(columns=columns, index=range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                column_text = column.get_text()
                df.iat[row_marker,column_marker] = column_text
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        if not (len(column_names) > 0):
            df.columns = df.iloc[0]
            if head:
                df = df.reindex(df.index.drop(0))
        return df

In [122]:
hp = HTMLTableParser()
du = url_dir[93]
manual_table = hp.parse_url_man(du[1]) # Grabbing the table from the tuple
manual_table = manual_table.rename(columns={"Batman":"Comic-book Title","608":"Issue","$2.25":"Price","DC":"Publisher","120,945":"Est. units"})
manual_df = filter_out_2nd_table(manual_table)
manual_df["date"] = du[0]
manual_df["url"] = du[1]
manual_df

,1,Comic-book Title,Issue,Price,Publisher,Est. units,date,url
0,1,Batman,608,$2.25,DC,"120,945",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
1,2,Ultimates,9,$2.25,Marvel,"110,000",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
2,3,Transformers War Within,1,$2.95,Dreamwave,"106,468",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
3,4,New X-Men,133,$2.25,Marvel,"104,932",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
4,5,Amazing Spider-Man,46,$2.25,Marvel,"104,835",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
5,6,Ultimate Spider-Man,28,$2.25,Marvel,"98,522",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
6,7,Ultimate Spider-Man,29,$2.25,Marvel,"98,365",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
7,8,Ultimate X-Men,23,$2.25,Marvel,"96,333",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
8,9,Spider-Man Black Cat Evil That Men\nDo,5,$2.99,Marvel,"95,160",2002-10-01,http://www.comichron.com/monthlycomicssales/20...
9,10,Uncanny X-Men,414,$2.25,Marvel,"89,536",2002-10-01,http://www.comichron.com/monthlycomicssales/20...


In [123]:
super_master_df = pandas.concat([master_df,manual_df],ignore_index=True)
super_master_df.describe()

,,,1,Comic-book Title,Est. units,Issue,Order Index,Price,Publisher,date,url
count,52252,20142,300,83038,77036,83038,6000,83038,83038,83038,83038
unique,397,471,300,18826,38050,2114,2991,130,651,271,271
top,253,118,118,Amazing Spider-Man,$3.99,1,2.20,$2.99,DC,2013-12-01,http://www.comichron.com/monthlycomicssales/20...
freq,173,61,1,405,99,11622,31,22752,22070,420,420


In [140]:
hp = HTMLTableParser()


def clean_share_df(df, col_name):
    mdf = populate_type_col(df,col_name)
    mdf = mdf.rename(columns = {col_name:"Publisher"})
    mdf = mdf[mdf['Publisher'].str.contains("Share")==False]
    mdf = mdf[mdf['Publisher'].str.len() > 0]
    return mdf

def urlt_to_df(d_u):
    url = d_u[1]
    print url
    fdf = hp.parse_url_share(url)
    fdf.columns = fdf.columns.str.strip()
    first_col = fdf.columns.values[0]
    second_col = fdf.columns.values[1]
    fdf = fdf.rename(columns = {second_col:"Share"})
    second_col = "Share"
    fdf[first_col] = fdf[first_col].str.strip()
    fdf["URL"] = url
    fdf["Share"] = fdf["Share"].str.strip()
    fdf["Date"] =d_u[0]
    fdf["Type"] = ""
    return clean_share_df(fdf,first_col)

def populate_type_col(df,col_val):
    x = 0
    cat = ""
    while x < len(df):
        if x == 0:
            df.set_value(x+1,"Type",col_val)
            cat = col_val
        else:
            val_c1 = df.iloc[x][col_val]
            if type(val_c1) != float:
                if ("Share" not in val_c1) and (val_c1 is not "") and ("Items" not in val_cl):
                    df.set_value(x+1,"Type",cat)
                else:
                    df.set_value(x+1,"Type",val_c1)
                    cat = val_c1
        x = x+1
    return df

In [141]:
fdf = hp.parse_url_share(url_dir[0][1])
print fdf

None
returning None
None


In [134]:
manual_share = []
i=0
master_share = urlt_to_df(url_dir[0])
for du_tup in url_dir[1:10]:
    try:
        share_df = urlt_to_df(du_tup)
        master_share = pd.concat([master_share,share_df],ignore_index=True)
    except:
        manual_share.append(du_tup)
        print du_tup
        print i
    i = i+1
    
#write_obj(master_share,"publisher_data.pickle")
#master_share.to_csv("publisher_data.csv")

master_share

http://www.comichron.com/monthlycomicssales/1995/1995-01Diamond.html


AttributeError: 'NoneType' object has no attribute 'columns'

In [84]:
import pandas
master_df = load_obj("top300.pickle")
master_df = master_df.drop(['Dollar sales','Dollars','Unit sales','Units'],inplace=False,axis=1,errors='ignore')
master_df

,,,Comic-book Title,Est. units,Issue,Order Index,Price,Publisher,date,url
0,NaN,1,The Amazing X-Men,NaN,1,229.5,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
1,NaN,2,The Astonishing X-Men,NaN,1,223.7,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
2,NaN,3,Weapon X,NaN,1,213.9,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
3,NaN,4,Mutants: Generation Next,NaN,1,194.1,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
4,NaN,5,Gambit & The X-Ternals,NaN,1,178.9,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
5,NaN,6,Factor-X,NaN,1,167.8,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
6,NaN,7,X-Man,NaN,1,159.4,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
7,NaN,8,Spawn,NaN,28,206.6,$1.95,Image,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
8,NaN,9,X-Calibre,NaN,1,147.2,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
9,NaN,10,X-Men: Chronicles,NaN,1,148.8,$3.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...


In [77]:
partial_df = master_df[master_df["Order Index"].isnull()]
partial_df = partial_df[partial_df["Est. units"].isnull()]
print partial_df.shape
partial_df

(389, 10)


,,,Comic-book Title,Est. units,Issue,Order Index,Price,Publisher,date,url
719,NaN,120,Classic Star\nWars: The Vandelhelm Mission,NaN,$2.50,NaN,Dark Horse,28.1,1995-03-01,http://www.comichron.com/monthlycomicssales/19...
1568,NaN,69,Star Wars: Jabba\nthe Hutt: The Hunger of Prin...,NaN,$2.50,NaN,Dark Horse,54.09,1995-06-01,http://www.comichron.com/monthlycomicssales/19...
4077,178,NaN,Star Wars: Jabba\nThe Hutt: The Gaar Suppoon Hit,NaN,$2.50,NaN,Dark Horse,8.60,1996-02-01,http://www.comichron.com/monthlycomicssales/19...
5371,272,NaN,Sherman's March\nThrough Atlanta to the Sea,NaN,$3.50,NaN,Heritage,2.20,1996-06-01,http://www.comichron.com/monthlycomicssales/19...
5575,176,NaN,Prelude to War:\nTwilight on the World of Magi...,NaN,$5.95,NaN,Acclaim,9.30,1996-07-01,http://www.comichron.com/monthlycomicssales/19...
6048,NaN,49,Star Wars Boba\nFett: When the Fat Lady Swings,NaN,$3.95,NaN,Dark Horse,"61,757",1996-09-01,http://www.comichron.com/monthlycomicssales/19...
6307,NaN,308,Morbid Angel:\nPenance Revised Color Special,NaN,$3.95,NaN,London Night,"3,943",1996-09-01,http://www.comichron.com/monthlycomicssales/19...
6691,338,NaN,Witchblade\nCollected Ed. Vol. 4 w/Custom Slip...,NaN,$10.95,NaN,Top Cow,"3,130",1996-10-01,http://www.comichron.com/monthlycomicssales/19...
6708,1,NaN,Superman: The\nWedding Album Collector's Edition,NaN,$4.95,NaN,DC,"227,686",1996-11-01,http://www.comichron.com/monthlycomicssales/19...
6797,90,NaN,Superman:\nWedding Album Collector's Ed. Next Day,NaN,$4.95,NaN,DC,"39,603",1996-11-01,http://www.comichron.com/monthlycomicssales/19...


In [81]:
def copy_value(df,x,to_copy,target_name):
    copy_val = df.iloc[x][to_copy]
    df.set_value(x,target_name,copy_val)
    return df

def transpose(df):
    i_arr = df.index
    for i in i_arr:
        url = df.loc[i]["url"]
        target_col = "Est. units"
        if "Diamond" in url:
            target_col = "Order Index"
        e_sales = df.loc[i]["Publisher"]
        df.set_value(i,target_col,e_sales)
        publisher = df.loc[i]["Price"]
        df.set_value(i,"Publisher",publisher)
        price = df.loc[i]["Issue"]
        df.set_value(i,"Price",price)
        df.set_value(i,"Issue","")
    return df

print partial_df.shape
partial_df = transpose(partial_df)

(389, 10)


In [85]:

master_df.update(partial_df)

In [124]:
#master_df.to_csv("top300limcols.csv",encoding="utf-8")
write_obj(super_master_df,"top300DONE.pickle")
super_master_df.to_csv("top300wmanual.csv",encoding="utf-8")

In [125]:
super_master_df

,,,1,Comic-book Title,Est. units,Issue,Order Index,Price,Publisher,date,url
0,NaN,1,NaN,The Amazing X-Men,NaN,1,229.5,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
1,NaN,2,NaN,The Astonishing X-Men,NaN,1,223.7,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
2,NaN,3,NaN,Weapon X,NaN,1,213.9,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
3,NaN,4,NaN,Mutants: Generation Next,NaN,1,194.1,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
4,NaN,5,NaN,Gambit & The X-Ternals,NaN,1,178.9,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
5,NaN,6,NaN,Factor-X,NaN,1,167.8,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
6,NaN,7,NaN,X-Man,NaN,1,159.4,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
7,NaN,8,NaN,Spawn,NaN,28,206.6,$1.95,Image,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
8,NaN,9,NaN,X-Calibre,NaN,1,147.2,$1.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...
9,NaN,10,NaN,X-Men: Chronicles,NaN,1,148.8,$3.95,Marvel,1995-01-01,http://www.comichron.com/monthlycomicssales/19...


In [78]:
print manual_share
master_share["Type"]
master_share.describe()

[(datetime.date(2017, 8, 1), 'http://www.comichron.com/monthlycomicssales/2017/2017-08.html')]


,Publisher,Share,URL,Date,Type
count,45826,45580,45826,45826,45826
unique,788,3377,271,271,49
top,Dark Horse,1,http://www.comichron.com/monthlycomicssales/20...,2002-12-01,Share of Overall Units
freq,1710,3043,317,317,7763


In [1]:
import pickle

def write_obj(obj,filename):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as handle:
        b = pickle.load(handle)
        return b

In [49]:
import pandas
import numpy as np

e = master_df["Order Index"]
e[e>0]

partial_df = master_df.truncate(before=6000)

index = partial_df['Est. units'].index[partial_df['Est. units'].isnull()]
index
print len(index)
for x in index[0:10]:
    print master_df.iloc[x]
#partial_df = master_df.drop(['Dollar sales','Dollars','Unit sales','Units'],inplace=False,axis=1,errors='ignore')
#partial_df.describe()

384
                                                                  NaN
\n                                                                 49
Comic-book Title       Star Wars Boba\nFett: When the Fat Lady Swings
Dollar sales                                                      NaN
Dollars                                                           NaN
Est. units                                                        NaN
Issue                                                           $3.95
Order Index                                                       NaN
Price                                                      Dark Horse
Publisher                                                      61,757
Unit sales                                                        NaN
Units                                                             NaN
date                                                       1996-09-01
url                 http://www.comichron.com/monthlycomicssales/19...
Name: 6048, dtyp

In [103]:
hp = HTMLTableParser()

def filter_out_2nd_table(df):
    title_col = "Comic-book Title"
    if title_col not in df.columns:
        title_col = "Comic-book\nTitle"
    p = df[title_col]
    i = p[p=="\n"]
    if len(i) > 0:
        i = i.index[0]
        return df.truncate(after=(i-1))
    else:
        i = p[p==""]
        if len(i) > 0:
            i = i.index[0]
            return df.truncate(after=(i-1))
        else:
            i = df[df[title_col].str.contains("Trade Paperback")]
            if len(i)>0:    
                i = i.index[0]
                return df.truncate(after=(i-1))
            else:
                return df
        
def get_comic_data(d_u):
    table = hp.parse_url(d_u[1]) # Grabbing the table from the tuple
    new_table = filter_out_2nd_table(table)
    if len(new_table) == len(table):
        print d_u
    new_table["date"] = d_u[0]
    new_table["url"] = d_u[1]
    return new_table

def get_comic_data_man(d_u):
    table = hp.parse_url(d_u[1],head=False) # Grabbing the table from the tuple
    
    
    new_table = filter_out_2nd_table(table)
    if len(new_table) == len(table):
        print d_u
    new_table["date"] = d_u[0]
    new_table["url"] = d_u[1]
    return new_table

#t = hp.parse_url(url_dir[24][1])
#filter_out_2nd_table(t)


In [59]:
#master_df = load_obj("top300.pickle")
#master_df.to_csv("top300.csv", encoding='utf-8')
print manual

[(datetime.date(2002, 10, 1), 'http://www.comichron.com/monthlycomicssales/2002/2002-10.html'), (datetime.date(2017, 8, 1), 'http://www.comichron.com/monthlycomicssales/2017/2017-08.html')]


In [19]:
76354+5995

82349

In [143]:
import numpy
other_df = get_comic_data(url_dir[39])
print url_dir[39]
n = numpy.where(other_df.columns.values == "Comic-book Title")
print n[0][0]
other_df.columns.values[n[0][0]+4]

(datetime.date(1998, 4, 1), 'http://www.comichron.com/monthlycomicssales/1998/1998-04.html')
(datetime.date(1998, 4, 1), 'http://www.comichron.com/monthlycomicssales/1998/1998-04.html')
2


u'Est. units'

In [103]:
master_df.iloc[12068]["url"]

'http://www.comichron.com/monthlycomicssales/1998/1998-04.html'

In [4]:
import pandas
master_df.to_csv("top300.csv")

In [12]:
print master_df["url"].iloc[87000]

master_df.iloc[87000]

http://www.comichron.com/monthlycomicssales/2017/2017-05.html


0
\n                                                                169
Comic-book Title                      Wolverine And Black Cat Claws 2
Issue                                                               3
Price                                                           $3.99
Publisher                                                      Marvel
Est. sales                                                     11,174
date                                                       2017-05-01
url                 http://www.comichron.com/monthlycomicssales/20...
Name: 87000, dtype: object

In [13]:
master_df.describe()

,,Comic-book Title,Issue,Price,Publisher,Est. sales,date,url
count,88128,88128,88128,88128,88128,88128,88128,88128
unique,324,294,83,13,26,323,272,272
top,118,Walking Dead Weekly,1,$2.99,Marvel,"3,973",1997-10-01,http://www.comichron.com/monthlycomicssales/20...
freq,272,1088,12512,41888,28288,544,324,324
